In [ ]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
import math
from operator import add
import builtins
from google.cloud import storage
from collections import defaultdict

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')



In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [ ]:
def open_gcp(file_name,dir_name):
    client = storage.Client(file_name)
    bucket = client.bucket(bucket_name)
    blob = bucket.get_blob(f'postings_gcp{dir_name}/' + file_name)
    return blob.open('rb')

def read_pickle(file_name,dir_name):
    stream = open_gcp(file_name+".pickle",dir_name)
    pick = pickle.load(stream)
    stream.close()
    return pick

def read_pkl(file_name,dir_name):
    stream = open_gcp(file_name+".pkl",dir_name)
    pick = pickle.load(stream)
    stream.close()
    return pick

In [ ]:
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
bucket_name = '318457645' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)

In [ ]:
from inverted_index_gcp import *

In [ ]:
inverted_text = read_pkl('index','')

In [ ]:
inverted_title = read_pkl('index_title','title')

In [ ]:
DL_dict = read_pickle('text_DL','')
inverted_text.DL = DL_dict

In [ ]:
term_total_dict = read_pickle('text_term_total','')
inverted_text.term_total = term_total_dict

In [ ]:
vec_len_dict = read_pickle('vec_len_total', '')
inverted_text.vec_len_doc = vec_len_dict

In [ ]:
TUPLE_SIZE = 6       
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w, base_dir):
    with closing(MultiFileReader()) as reader:
        locs = inverted.posting_locs[w]
        b = reader.read(locs, inverted.df[w] * TUPLE_SIZE, base_dir)
        posting_list = []
        for i in range(inverted.df[w]):
            doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
            tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
            posting_list.append((doc_id, tf))
        return posting_list

In [ ]:
def generate_query_tfidf_vector(query_to_search,index):
    
    dict_term_tfidf = {}
    counter = Counter(query_to_search)
    for term in query_to_search:
        if term in index.term_total.keys(): #avoid terms that do not appear in the index.               
            tf = counter[term]/len(query_to_search) # term frequency divded by the length of the query
            df = index.df[term]            
            idf = math.log((len(index.DL))/(df),10) #smoothing
            dict_term_tfidf[term] = tf*idf
    return dict_term_tfidf

In [ ]:
 def get_candidate_documents_and_scores(query_to_search,index,base_dir):

    candidates = {}
    N = len(index.DL)
    for term in np.unique(query_to_search):  
        if term in index.df.keys():
            list_of_doc = read_posting_list(index, term, base_dir)
            normlized_tfidf = [(doc_id,(freq/index.DL[doc_id])*math.log(N/index.df[term],10)) for doc_id, freq in list_of_doc]           
            for doc_id, tfidf in normlized_tfidf:
                candidates[(doc_id,term)] = candidates.get((doc_id,term), 0) + tfidf               
    return candidates

In [ ]:
def generate_cosine_dict(query_to_search,index,base_dir):
    
    
    cosine_dict = {}
    candidates_scores = get_candidate_documents_and_scores(query_to_search,index,base_dir) #We do not need to utilize all document. Only the docuemnts which have corrspoinding terms with the query.
    dict_query = generate_query_tfidf_vector(query_to_search,index)
    for key in candidates_scores:
        doc_id, term = key
        cosine_dict[doc_id] = cosine_similarity(candidates_scores[key],dict_query,index,doc_id) 
    return cosine_dict

In [ ]:
def cosine_similarity(tfidf_doc,dict_query,index,doc_id):
    
    #for one query and d
    cos_sim_mone = 0.0
    query_size_vec = 0.0
    for term in dict_query.keys():
        cos_sim_mone += dict_query[term] * tfidf_doc
        query_size_vec += math.pow(dict_query[term],2)
    cos_sim_total = cos_sim_mone / (index.vec_len_doc[doc_id] * (math.sqrt(query_size_vec)))
    return cos_sim_total

In [ ]:
def get_top_n(query,index, base_dir,N=3):
    
#     vec_query = generate_query_tfidf_vector(query,index)
    sim_dict = generate_cosine_dict(query,index, base_dir)
    lst = [(doc_id , builtins.round(score,5)) for doc_id, score in sim_dict.items()]
    return sorted(lst, key = lambda x: x[1], reverse=True)[:N]
    

In [ ]:
dict_q = {}
dict_q[1] = ['python']
tfidf_queries_score_train = get_top_n(['python'], inverted_text,'' , 100)
print(tfidf_queries_score_train)

# lst = [57069491, 65967176, 42163310, 878659, 27306717, 41677925, 1074657, 44240443, 17296107, 60952488, 43603241, 22114132, 46208997, 36450985, 41974555, 56289672, 60616450, 59502488, 33038861, 61699239, 61651800, 39368416, 29129051, 55935213, 54537218, 62372638, 60774345, 63090183, 37497391, 51430647, 67229718, 9110929, 61329320, 44254295, 41974496, 200563, 58481694, 48530084, 56289572, 22144990, 612052, 59162931, 55511148, 55511147, 61073786, 59892, 36484005, 36484254, 66423851, 62482816, 1275470, 5676692, 7927053, 60754840, 26999426, 60744481, 56289553, 60463979, 701741, 60283633, 1129847, 36439749, 4451883, 55511155, 22144721, 45359871, 723126, 43655965, 57275457, 12673434, 43867095, 26763420, 39293265, 15003874, 41668588, 61592102, 67063919, 11891433, 64057670, 61513780, 39345917, 67063906, 1221476, 41008758, 60587000, 7729, 2152196, 5027882, 509738, 403585, 26866372, 1339248, 3473503, 4148655]
lst = [23862, 23329, 53672527, 21356332, 4920126, 5250192, 819149, 46448252, 83036, 88595, 18942, 696712, 2032271, 1984246, 5204237, 645111, 18384111, 3673376, 25061839, 271890, 226402, 2380213, 1179348, 15586616, 50278739, 19701, 3596573, 4225907, 19160, 1235986, 6908561, 3594951, 18805500, 5087621, 25049240, 2432299, 381782, 9603954, 390263, 317752, 38007831, 2564605, 13370873, 2403126, 17402165, 23678545, 7837468, 23954341, 11505904, 196698, 34292335, 52042, 2247376, 15858, 11322015, 13062829, 38833779, 7800160, 24193668, 440018, 54351136, 28887886, 19620, 23045823, 43003632, 746577, 1211612, 8305253, 14985517, 30796675, 51800, 964717, 6146589, 13024, 11583987, 57294217, 27471338, 5479462]
lst_docs_returned = [x[0] for x in tfidf_queries_score_train]

lst_equal = [x for x in lst_docs_returned if x in lst]
print(len(lst))
print(len(lst_equal))

lst_indecies = [lst_docs_returned.index(x) for x in lst if x in lst_docs_returned]
print(lst_indecies)